In [ ]:
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

In [ ]:
import tensorflow as tf

In [ ]:
sess = tf.InteractiveSession() #式の作成と実行を交互に行うのでこれを指定する

In [ ]:
#器の用意
x = tf.placeholder(tf.float32, shape = [None, 784])
y_ = tf.placeholder(tf.float32, shape = [None, 10])

In [ ]:
#重み。行列を渡すと初期化して返す
def weight_variable(shape):
    #truncated_normal:正規分布の左右を切り取ったもののイメージ
    #stddev:標準偏差
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

In [ ]:
#バイアス
def bias_variable(shape):
    #0だと計算が進まないのでバイアス値は少し
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [ ]:
#たたみ込み処理の定義
def conv2d(x, W):
    #strides = [1,1,1,1]:1pix毎に処理する事
    #padding = 'SAME':左右に0を足したものにする
    #畳込みを行う関数:tf.nn.conv2d
    return tf.nn.conv2d(x, W, strides = [1,1,1,1], padding = 'SAME')

In [ ]:
#たたみ込んだ画像をsizeを小さくして特徴を抽出。一番sizeの大きなデータをかえす
def max_pool_2x2(x):
    #ksize適用数範囲2*2
    #strides 2*2ずらして処理
    return tf.nn.max_pool(x, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

In [ ]:
#NNを記載していく
#1層目
#重みというかFilter。5*5の画像に1個のデータが32枚
W_conv1 = weight_variable([5, 5, 1, 32])
#bias
b_conv1 = bias_variable([32])
#入力画像の格納。-1は実行時に自動的に値を設定してくれるらしい
x_image = tf.reshape(x, [-1, 28, 28, 1])
#フィルタをかけた値にReLUの変換をした値
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
#プーリング。次元圧縮。今回はMax poolingを採用
h_pool1 = max_pool_2x2(h_conv1)

In [ ]:
#2層目
#Filter
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])
#一層目の値を上の値をReLUにかける
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
#プーリング
h_pool2 = max_pool_2x2(h_conv2)

In [ ]:
#全結合層
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])

h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [ ]:
#dropout処理 オーバーフィッティングを避ける処理
#それを避ける用の変数
keep_prob = tf.placeholder(tf.float32)

#dropout処理後の値を格納
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [ ]:
#読み出し層
#10:は10種類の意図
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) * b_fc2
#ここまでで層の記載は完

In [ ]:
#バックプロパゲーション準備の為の損失関数クロスエントロピーの定義
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_conv, labels = y_))

#AdamOptimizerってやつで最小化させる
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

#正解の値
#argmaxは一番大きい物をとってっくる：つまり確率の一番大きい物
correct_rediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))

#結果確認用
accuracy = tf.reduce_mean(tf.cast(correct_rediction, tf.float32))

#パラメータのリセット
sess.run(tf.global_variables_initializer())

for i in range (200):
    #トレーニングデータからランダムに50個データを持ってくる
    batch = mnist.train.next_batch(50)
    
    #100エポック事に結果を出力
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict = {x:batch[0], y_:batch[1], keep_prob:1.0})
        print("step %d, traning accuracy %g"%(1, train_accuracy))
    #トレーニング
    train_step.run(feed_dict = {x:batch[0], y_:batch[1], keep_prob:0.5})

print("test accuracy %g"%accuracy.eval(feed_dict = {x:mnist.test.images, y_:mnist.test.labels, keep_prob:1.0}))